Importing Modules

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping

This accuracy function calculates accuracy given predicted and actual labels


In [ ]:
def accuracy(ypredicted,ytest):
  counter=0
  for i in range(len(ypredicted)):
    if(ypredicted.iloc[i,0]==ytest.iloc[i,0]):
      counter+=1
  return counter/len(ypredicted)

The first line scales the data and i made two different datasets X and Y,Y containing the labels and X containing the features


In [ ]:
scaler=StandardScaler()
nparr =np.load('/content/QG_jets.npz')
datasetXarr = nparr["X"]
datasetYarr = nparr["y"]
# datasetX = pd.DataFrame(datasetX)
dfY = pd.DataFrame(datasetYarr)
dfY.head()

,0
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0


The datasetXarr is a 3d array it consists of 1 lakh rows each row is one jet stream the jet stream consists of some electrons and 4 features are given for each electron ,I found the average of each feature for all electrons contained in each jet stream(ignoring the values which were null)

In [ ]:

maskKarneKeBaad = np.ma.masked_equal(datasetXarr,0)
maskKeBaadkaMean = maskKarneKeBaad.mean(axis=1)
meanarr = maskKeBaadkaMean.filled(np.nan)
dfX=pd.DataFrame(meanarr)
dfX.head()

,0,1,2,3
0,27.823181,-0.017726,4.715097,-8.777778
1,31.282841,0.166467,3.020627,-9.882353
2,8.896696,-1.377024,0.487845,-64.315789
3,13.703112,0.666248,5.545959,78.567568
4,15.118487,-0.388895,4.536264,5.942857


This block of codes just splits the data into training and testing data I (i took 80% of data for training and 20% of data for testing)

In [ ]:
dfXTrain=dfX.iloc[:80000,:]
dfXtest=dfX.iloc[80000:,:]
dfYTrain=dfY.iloc[:80000,:]
dfYtest=dfY.iloc[80000:,:]
dfXTrainScaled=scaler.fit_transform(dfXTrain)
dfXtestScaled=scaler.transform(dfXtest)

Made a neural network to predict the labels it has a simple architecture with activation function of last perceptron being sigmoid(since this is a binary classification problem) and i used some techniques such as learning rate scheduler ,early stopping and Dropout layers to prevent overfitting on the training data

In [ ]:

model=models.Sequential()
lrScheduler=ReduceLROnPlateau(monitor='val_loss',factor=0.5,patience=5,verbose=1)
early_stopping=EarlyStopping(monitor='val_loss',patience=10,verbose=1,restore_best_weights=True)
# model.add(layers.Dense(64,activation='relu',input_dim=4))
model.add(layers.Dense(32,activation='relu',input_dim=4))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                     │ (None, 32)                  │             160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 705 (2.75 KB)

 Trainable params: 705 (2.75 KB)

 Non-trainable params: 0 (0.00 B)

Trained the model


In [ ]:
model.fit(dfXTrainScaled,dfYTrain,epochs=10,callbacks=[lrScheduler,early_stopping])

Epoch 1/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.6937 - loss: 0.5843 - learning_rate: 0.0010
Epoch 2/10
 107/2500 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.7564 - loss: 0.5298

/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/callback_list.py:96: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)
/usr/local/lib/python3.11/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate
  current = self.get_monitor_value(logs)


2500/2500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7643 - loss: 0.5213 - learning_rate: 0.0010
Epoch 3/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7704 - loss: 0.5147 - learning_rate: 0.0010
Epoch 4/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7661 - loss: 0.5130 - learning_rate: 0.0010
Epoch 5/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7677 - loss: 0.5125 - learning_rate: 0.0010
Epoch 6/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7653 - loss: 0.5121 - learning_rate: 0.0010
Epoch 7/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7648 - loss: 0.5141 - learning_rate: 0.0010
Epoch 8/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.7662 - loss: 0.5127 - learning_rate: 0.0010
Epoch 9/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7680 - loss: 0.5107 - learning_rate: 0.0010
Epoch 10/10
2500/2500 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.7663 - loss: 0.5102 - learning_rate: 0.0010


Tested the model on testing data and the second line of this code just converts the values greater than 0.5 to 1 and less than 0.5 to 0 (since it was binary classification)

In [ ]:
prediction =model.predict(dfXtestScaled)
finalPrediction=np.where(prediction>0.5,1,0)

625/625 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


Got accuracy of 76.75 percent on testing data

In [ ]:
dfYpredicted = pd.DataFrame(finalPrediction)
100*accuracy(dfYpredicted,dfYtest)

76.755